In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# customisations - ensure tables show all columns
#pd.set_option("max_columns", 100)

page = 29
packetSize = 10

Next, we define a general, all-purpose function to process get requests from an API, supplied through a URL parameter. A dictionary of parameters can be supplied which is passed into the get request automatically, depending on the requirements of the API.

Rather than simply returning the response, we handle a couple of scenarios to help automation. Occasionally we encounter an SSL Error, in which case we simply wait a few seconds then try again (by recursively calling the function). When this happens, and generally throughout this project, we provide quite verbose feedback to show when these errors are encountered and how they are handled.

Sometimes there is no response when a request is made (returns None). This usually happens when too many requests are made in a short period of time, and the polling limit has been reached. We try to avoid this by pausing briefly between requests, as we'll see later, but in case we breach the polling limit we wait 10 seconds then try again.

Handling these errors in this way ensures that our function almost always returns the desired response, which we return in json format to make processing easier.

In [2]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except requests.exceptions.SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 60 seconds...')
        time.sleep(60)
        print('Retrying.')
        return get_request(url, parameters)

In [3]:
response_0 = requests.get(f"https://steamspy.com/api.php?request=all&page={page}")

response_0

<Response [200]>

## Responses

In [4]:
steam_spy_all = pd.DataFrame.from_dict(response_0.json(), orient='index')

# generate sorted app_list from steamspy data
app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)

# export disabled to keep consistency across download sessions
app_list.to_csv(f'./data/{page}/app_list_{page}.csv', index=False)

# instead read from stored csv
app_list = pd.read_csv(f'./data/{page}/app_list_{page}.csv')

# display first few rows
app_list.head()


,appid,name
0,10520,Beijing 2008 - The Official Video Game of the ...
1,15920,Luxor 2
2,47570,Mishap 2: An Intentional Haunting - Collector'...
3,260040,YOU DON'T KNOW JACK TELEVISION
4,276360,Riding Star - Horse Championship!


## Define Download Logic

Now we have the `app_list` dataframe, we can iterate over the app IDs and request individual app data from the servers. Here we set out our logic to retrieve and process this information, then finally store the data as a csv file.

Because it takes a long time to retrieve the data, it would be dangerous to attempt it all in one go as any errors or connection time-outs could cause the loss of all our data. For this reason we define a function to download and process the requests in batches, appending each batch to an external file and keeping track of the highest index written in a separate file.

This not only provides security, allowing us to easily restart the process if an error is encountered, but also means we can complete the download across multiple sessions.

Again, we provide verbose output for rows exported, batches complete, time taken and estimated time remaining.

In [5]:
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=packetSize, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

Next we define some functions to handle and prepare the external files.

We use `reset_index` for testing and demonstration, allowing us to easily reset the index in the stored file to 0, effectively restarting the entire download process.

We define `get_index` to retrieve the index from file, maintaining persistence across sessions. Every time a batch of information (app data) is written to file, we write the highest index within `app_data` that was retrieved. As stated, this is partially for security, ensuring that if there is an error during the download we can read the index from file and continue from the end of the last successful batch. Keeping track of the index also allows us to pause the download, continuing at a later time.

Finally, the `prepare_data_file` function readies the csv for storing the data. If the index we retrieved is 0, it means we are either starting for the first time or starting over. In either case, we want a blank csv file with only the header row to begin writing to, se we wipe the file (by opening in write mode) and write the header. Conversely, if the index is anything other than 0, it means we already have downloaded information, and can leave the csv file alone.

In [6]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

## Download Steam Data

Now we are ready to start downloading data and writing to file. We define our logic particular to handling the steam API - in fact if no data is returned we return just the name and appid - then begin setting some parameters. We define the files we will write our data and index to, and the columns for the csv file. The API doesn't return every column for every app, so it is best to explicitly set these.

Next we run our functions to set up the files, and make a call to `process_batches` to begin the process. Some additional parameters have been added for demonstration, to constrain the download to just a few rows and smaller batches. Removing these would allow the entire download process to be repeated.

In [7]:
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data

## Download Steam Data - File Parameters replication

In [8]:
# Set file parameters
download_path = f'./data/{page}'
steam_app_data = f'steam_app_data_{page}.csv'
steam_index = f'steam_index_{page}.txt'

In [9]:
steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    #end=10,
    batchsize=packetSize
)

Starting at index 0:

Exported lines 0-9 to steam_app_data_29.csv. Batch 0 time: 0:00:15 (avg: 0:00:15, remaining: 0:24:16)
Exported lines 10-19 to steam_app_data_29.csv. Batch 1 time: 0:00:14 (avg: 0:00:14, remaining: 0:23:55)
Exported lines 20-29 to steam_app_data_29.csv. Batch 2 time: 0:00:15 (avg: 0:00:15, remaining: 0:23:44)
Exported lines 30-39 to steam_app_data_29.csv. Batch 3 time: 0:00:14 (avg: 0:00:14, remaining: 0:23:25)
Exported lines 40-49 to steam_app_data_29.csv. Batch 4 time: 0:00:15 (avg: 0:00:15, remaining: 0:23:13)
Exported lines 50-59 to steam_app_data_29.csv. Batch 5 time: 0:00:15 (avg: 0:00:15, remaining: 0:23:03)
Exported lines 60-69 to steam_app_data_29.csv. Batch 6 time: 0:00:16 (avg: 0:00:15, remaining: 0:23:01)
Exported lines 70-79 to steam_app_data_29.csv. Batch 7 time: 0:00:15 (avg: 0:00:15, remaining: 0:22:47)
Exported lines 80-89 to steam_app_data_29.csv. Batch 8 time: 0:00:14 (avg: 0:00:15, remaining: 0:22:28)
Exported lines 90-99 to steam_app_data_29.cs

In [10]:
# inspect downloaded data
pd.read_csv(f'./data/{page}/steam_app_data_{page}.csv').head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Beijing 2008™ - The Official Video Game of the...,10520,0.0,False,NaN,NaN,<p> Embrace the competi...,<p> Embrace the competi...,Embrace the competitive spirit of the world's ...,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '18', 'description': 'Sports'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,NaN,NaN,"{'coming_soon': False, 'date': 'Aug 14, 2008'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
1,game,Luxor 2,15920,0.0,False,NaN,NaN,<p>The much anticipated sequel to the #1 casua...,<p>The much anticipated sequel to the #1 casua...,The much anticipated sequel to the #1 casual g...,...,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,NaN,NaN,"{'coming_soon': False, 'date': 'Jun 10, 2008'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
2,game,Mishap 2: An Intentional Haunting - Collector'...,47570,0.0,False,NaN,NaN,<p>This time it’s no Mishap—it’s Intentional!<...,<p>This time it’s no Mishap—it’s Intentional!<...,Join the Mishap gang to help Milton Hobblepop ...,...,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,NaN,NaN,"{'coming_soon': False, 'date': 'Jan 28, 2011'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,game,YOU DON'T KNOW JACK TELEVISION,260040,0.0,False,NaN,NaN,"No, this is not a video of the YDKJ TV show th...","No, this is not a video of the YDKJ TV show th...","No, this is not a video of the YDKJ TV show th...",...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 2030044, 'name': 'YDKJ_TV', 'thumbnail...",NaN,NaN,"{'coming_soon': False, 'date': 'Nov 5, 2013'}",{'url': 'https://jellyvisiongames.zendesk.com/...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
4,game,Riding Star - Horse Championship!,276360,0.0,False,NaN,NaN,<h1>🔥 Upcoming games - Wishlist now!</h1><p><a...,"<img src=""https://cdn.akamai.steamstatic.com/s...",Ride like a world champion! Breathtaking actio...,...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256683675, 'name': 'Riding Star Traile...",NaN,NaN,"{'coming_soon': False, 'date': 'Mar 27, 2014'}","{'url': '', 'email': 'info@mut.de'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"


## Download SteamSpy data

To retrieve data from SteamSpy we perform a very similar process. Our parse function is a little simpler because of the how data is returned, and the maximum polling rate of this API is higher so we can set a lower value for `pause` in the `process_batches` function and download more quickly. Apart from that we set the new variables and make a call to the `process_batches` function once again.

In [11]:
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}
    
    json_data = get_request(url, parameters)
    return json_data

## Download SteamSpy Data path replication

In [12]:
# set files and columns
download_path = f'./data/{page}'
steamspy_data = f'steamspy_data_{page}.csv'
steamspy_index = f'steamspy_index_{page}.txt'

In [13]:
steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
    #end=20,
    batchsize=packetSize,
    pause=0.3
)

Starting at index 0:

Exported lines 0-9 to steamspy_data_29.csv. Batch 0 time: 0:00:10 (avg: 0:00:10, remaining: 0:17:17)
Exported lines 10-19 to steamspy_data_29.csv. Batch 1 time: 0:00:10 (avg: 0:00:10, remaining: 0:16:57)
Exported lines 20-29 to steamspy_data_29.csv. Batch 2 time: 0:00:10 (avg: 0:00:10, remaining: 0:16:41)
Exported lines 30-39 to steamspy_data_29.csv. Batch 3 time: 0:00:10 (avg: 0:00:10, remaining: 0:16:28)
Exported lines 40-49 to steamspy_data_29.csv. Batch 4 time: 0:00:10 (avg: 0:00:10, remaining: 0:16:17)
Exported lines 50-59 to steamspy_data_29.csv. Batch 5 time: 0:00:10 (avg: 0:00:10, remaining: 0:16:06)
Exported lines 60-69 to steamspy_data_29.csv. Batch 6 time: 0:00:10 (avg: 0:00:10, remaining: 0:15:55)
Exported lines 70-79 to steamspy_data_29.csv. Batch 7 time: 0:00:10 (avg: 0:00:10, remaining: 0:15:43)
Exported lines 80-89 to steamspy_data_29.csv. Batch 8 time: 0:00:10 (avg: 0:00:10, remaining: 0:15:33)
Exported lines 90-99 to steamspy_data_29.csv. Batch 9

In [14]:
# inspect downloaded steamspy data
pd.read_csv(f'./data/{page}/steamspy_data_{page}.csv').head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10520,Beijing 2008 - The Official Video Game of the ...,Eurocom,SEGA,NaN,0,1,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,English,Sports,0,{'Sports': 53}
1,15920,Luxor 2,MumboJumbo,MumboJumbo,NaN,13,20,0,"0 .. 20,000",1,0,2,0,999.0,999.0,0.0,"English, French, German",Casual,0,"{'Casual': 26, 'Match 3': 7}"
2,47570,Mishap 2: An Intentional Haunting - Collector'...,Virtual Prophecy Entertainment,"Namco Networks America, Inc.",NaN,2,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,English,Casual,0,"{'Casual': 20, 'Hidden Object': 6}"
3,260040,YOU DON'T KNOW JACK TELEVISION,"Jackbox Games, Inc.","Jackbox Games, Inc.",NaN,32,18,0,"0 .. 20,000",2,0,4,0,299.0,299.0,0.0,English,"Casual, Indie",0,"{'Trivia': 39, 'Casual': 26, 'Indie': 21, 'Par..."
4,276360,Riding Star - Horse Championship!,Sproing,familyplay,NaN,63,30,0,"0 .. 20,000",2049,0,2049,0,999.0,999.0,0.0,"English, French, Italian, German, Spanish - Sp...","Casual, Simulation, Sports",0,"{'Casual': 61, 'Simulation': 48, 'Sports': 41,..."
